In [1]:
from basic import *
import json

years = [2010, 2012, 2014, 2016, 2018]
urbans_postfix = ['', '12', '14', '16', '18']

计算恩格尔系数(Engel Coefficient)

在家庭表格中，经查阅json得，总支出为`expense`, `expense`, `fexp`, `fexp`, `fexp`，食品支出在各年均为`food`

In [2]:
family_numbers = multicount('famecon')

rural_numbers = multicount(
                           'famecon',
                           conditions="urban{year}=0",
                           condition_format=['', '12', '14', '16', '18']
                          )

urban_numbers = multicount(
    'famecon',
    conditions='urban{year}=1',
    condition_format=urbans_postfix
)

incs = ['faminc', 'fincome1', 'finc', 'finc', 'finc']
exps = ['expense', 'expense', 'fexp', 'fexp', 'fexp']

expenses = []
expenses.append(csum('famecon_2010', 'expense', 'expense'))
expenses.append(csum('famecon_2012', 'expense', 'expense'))
expenses.append(csum('famecon_2014', 'fexp', 'fexp>0'))
expenses.append(csum('famecon_2016', 'fexp', 'fexp>0'))
expenses.append(csum('famecon_2018', 'fexp', 'fexp>0'))

incomes = [csum('famecon_'+str(years[i]), incs[i], incs[i]+'>0') for i in range(0, 5)]

foods = [csum('famecon_' + str(years[i]), 'food', 'food>0') for i in range(0, 5)]

print(expenses, incomes, foods)

[402542365.075531, 486836722.57928467, Decimal('497702764'), Decimal('571369774'), Decimal('615527031')] [496442938.26553345, 514645643.0, Decimal('682036119'), Decimal('827876736'), 907412956.0] [120780539.41347122, 186537850.45031738, 220333666.67297363, 250179096.0, 281738556.0]


In [3]:
engels = [food / float(expense) for food, expense in zip(foods, expenses)]

engels2 = [food / float(income) for food, income in zip(foods, incomes)]

print(engels, engels2)

[0.30004429320329695, 0.3831630643268461, 0.4427013121288867, 0.4378584716663714, 0.4577192256565577] [0.2432918873525583, 0.36245881605630803, 0.3230527834743216, 0.30219365410456467, 0.3104854896958293]


In [4]:
# urban

rural_expenses = [int(csum('famecon_'+str(years[i]), exps[i], exps[i]+'>0 and urban' + urbans_postfix[i] + '=0')) for i in range(0, 5)]
rural_foods = [int(csum('famecon_' + str(years[i]), 'food', 'food>0 and urban' + urbans_postfix[i] + '=0')) for i in range(5)]
rural_engels = [food / expense for food, expense in zip(rural_foods, rural_expenses)]

urban_expenses = [int(csum('famecon_'+str(years[i]), exps[i], exps[i]+'>0 and urban' + urbans_postfix[i] + '=1')) for i in range(0, 5)]
urban_foods = [int(csum('famecon_' + str(years[i]), 'food', 'food>0 and urban' + urbans_postfix[i] + '=1')) for i in range(5)]
urban_engels = [food / expense for food, expense in zip(urban_foods, urban_expenses)]

print(rural_expenses, rural_foods)
print(rural_engels)
print(urban_expenses, urban_foods)
print(urban_engels)

[161285654, 200781361, 209061906, 200682397, 217168141] [48504844, 86710937, 79260579, 83170500, 93889752]
[0.3007387377429117, 0.4318674630360734, 0.37912492293072275, 0.41443844225161414, 0.43233667501901213]
[241256710, 279227030, 284431288, 353659495, 389373698] [72275695, 97961413, 139095367, 159427796, 183601404]
[0.2995800406960702, 0.3508306950082877, 0.48902976876439835, 0.45079461531210974, 0.47153006210501663]


计算收入，各年总收入的列名（属性名）分别为：`faminc`,`fincome1`,`finc`,`finc`,`finc`

利用收入和支出可以计算收支比

In [5]:
incomes = [int(csum('famecon_'+str(years[i]), incs[i], incs[i] + '>0')) for i in range(0,5)]

print('total incomes:')
print(incomes)

urban_incomes = [int(csum('famecon_'+str(years[i]), incs[i], urban_cond(i))) for i in range(0, 5)]

rural_incomes = [int(csum('famecon_' + str(years[i]), incs[i], urban_cond(i, is_urban=False))) for i in range(0, 5)]

print('r1: urban and rural, r2: incomes and expenses:')
print(urban_incomes, rural_incomes)
print(urban_expenses, rural_expenses)

print('average incomes and expenses')
print(ratio(urban_incomes, urban_numbers, '.0f'), ratio(rural_incomes, rural_numbers, '.0f'))
print(ratio(urban_expenses, urban_numbers, '.0f'), ratio(rural_expenses, rural_numbers, '.0f'))

# ie_ratios for the ratio of incomes and expenses.
ie_ratios = [float(income / expense) for income, expense in zip(incomes, expenses)]
urban_ie_ratios = ratio(rural_incomes, rural_expenses)
rural_ie_ratios = ratio(urban_incomes, urban_expenses)

print('ratios, total, urban and rural')
myprint(ie_ratios)
print(repr(urban_ie_ratios).replace('\'',''))
myprint(rural_ie_ratios)
     

total incomes:
[496442938, 514645643, 682036119, 827876736, 907412956]
r1: urban and rural, r2: incomes and expenses:
[303118699, 296017975, 405938343, 505856044, 594606412] [193324239, 213092286, 270869862, 294599121, 297483406]
[241256710, 279227030, 284431288, 353659495, 389373698] [161285654, 200781361, 209061906, 200682397, 217168141]
average incomes and expenses
['42675', '48929', '59794', '73132', '82060'] ['25127', '29903', '38465', '43438', '44059']
['33965', '46153', '41896', '51129', '53736'] ['20963', '28176', '29688', '29590', '32164']
ratios, total, urban and rural
[1.2332687962094373, 1.0571216572845663, 1.3703683570461345, 1.4489333767242647, 1.4742048850816432]
[1.20, 1.06, 1.30, 1.47, 1.37]
[1.26, 1.06, 1.43, 1.43, 1.53]


以下计算一些消费性支出，属性列用`expenses_list`储存

这些属性列分布对应：食品、衣服、居住、日用品、医疗保健、交通、文教娱乐、其他支出。

计算消费比例，化成百分数。

In [14]:
expenses_list = ['food', 'dress', 'house', 'daily', 'med', 'trco', 'eec', 'other']

expenses_list_zh_cn = ['食品', '衣服', '居住', '日用品', '医疗保健', '交通', '文教娱乐', '其他支出']

urban_expenses_2010 = [int(csum('famecon_2010', attr, 'urban=1')) for attr in expenses_list]
rural_expenses_2010 = [int(csum('famecon_2010', attr, 'urban=0')) for attr in expenses_list]

urban_expenses_2018 = [int(csum('famecon_2018', attr, 'urban18=1')) for attr in expenses_list]
rural_expenses_2018 = [int(csum('famecon_2018', attr, 'urban18=0')) for attr in expenses_list]

urban_average_expenses_2010 = ratio(urban_expenses_2010, urban_numbers[0])
rural_average_expenses_2010 = ratio(rural_expenses_2010, rural_numbers[0])

urban_average_expenses_2018 = ratio(urban_expenses_2018, urban_numbers[4])
rural_average_expenses_2018 = ratio(rural_expenses_2018, rural_numbers[4])

myprint(urban_average_expenses_2018)
myprint(rural_average_expenses_2018)

# 输出为echart饼图的格式
dict_urban10 = [dict([('value', float(value)), ('name', name)]) for value, name in zip(urban_average_expenses_2010, expenses_list_zh_cn)]
dict_rural10 = [dict([('value', float(value)), ('name', name)]) for value, name in zip(rural_average_expenses_2010, expenses_list_zh_cn)]
dict_urban18 = [dict([('value', float(value)), ('name', name)]) for value, name in zip(urban_average_expenses_2018, expenses_list_zh_cn)]
dict_rural18 = [dict([('value', float(value)), ('name', name)]) for value, name in zip(rural_average_expenses_2018, expenses_list_zh_cn)]
print(dict_urban10)
print(dict_rural10)
print(dict_urban18)
print(dict_rural18)

[25338.31, 3921.81, 12209.89, 11279.43, 6624.12, 6207.07, 8647.46, 1756.77]
[13905.47, 2292.99, 7849.68, 6783.63, 5677.98, 4458.00, 4327.68, 689.10]
[{'value': 10175.38, 'name': '食品'}, {'value': 1582.66, 'name': '衣服'}, {'value': 2460.22, 'name': '居住'}, {'value': 3716.38, 'name': '日用品'}, {'value': 3765.21, 'name': '医疗保健'}, {'value': 4216.56, 'name': '交通'}, {'value': 4176.03, 'name': '文教娱乐'}, {'value': 689.83, 'name': '其他支出'}]
[{'value': 6304.24, 'name': '食品'}, {'value': 767.54, 'name': '衣服'}, {'value': 937.18, 'name': '居住'}, {'value': 1966.84, 'name': '日用品'}, {'value': 3141.09, 'name': '医疗保健'}, {'value': 2300.2, 'name': '交通'}, {'value': 2421.02, 'name': '文教娱乐'}, {'value': 910.24, 'name': '其他支出'}]
[{'value': 25338.31, 'name': '食品'}, {'value': 3921.81, 'name': '衣服'}, {'value': 12209.89, 'name': '居住'}, {'value': 11279.43, 'name': '日用品'}, {'value': 6624.12, 'name': '医疗保健'}, {'value': 6207.07, 'name': '交通'}, {'value': 8647.46, 'name': '文教娱乐'}, {'value': 1756.77, 'name': '其他支出'}]
[{'value': 1